In [1]:
import json
import os
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from utils.general import load_video_graph
from utils.chat_api import generate_messages, get_response_with_retry
from retrieve import answer_with_retrieval
from prompts import prompt_agent_verify_answer


def process_qa(qa):
    mem = load_video_graph(qa["mem_path"])
    question = qa["question"]
    agent_answer, session = answer_with_retrieval(mem, question)
    qa["agent_answer"] = agent_answer
    qa["session"] = session
    return qa


def verify_qa(qa):
    questions = qa["question"]
    ground_truth = qa["answer"]
    agent_answer = qa["agent_answer"]
    qa_sample = {
        "question": questions,
        "ground_truth_answer": ground_truth,
        "agent_answer": agent_answer,
    }

    input = [
        {
            "type": "text",
            "content": json.dumps(qa_sample),
        },
        {
            "type": "text",
            "content": prompt_agent_verify_answer,
        },
        {
            "type": "text",
            "content": "Now answer if the answer from the baseline is correct or not:",
        },
    ]
    messages = generate_messages(input)
    model = "gpt-4o-2024-11-20"
    response = get_response_with_retry(model, messages)
    qa["verify_result"] = response[0]

    return qa


def process_qa_list(qa_list):
    for qa in qa_list:
        process_qa(qa)
    max_workers = 16
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        new_qa_list = list(tqdm(executor.map(process_qa, qa_list), total=len(qa_list), desc="Generating answers"))
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        new_qa_list = list(tqdm(executor.map(verify_qa, new_qa_list), total=len(qa_list), desc="Verifying answers"))
    return new_qa_list

In [2]:
qa_list = []
dataset = "data/annotations/small_train.jsonl"
dataset_with_agent_answer = "data/annotations/small_train_with_agent_answer.jsonl"

with open(dataset, "r") as f:
    for line in f:
        qa = json.loads(line)
        if os.path.exists(qa["mem_path"]):
            qa_list.append(qa)

new_qa_list = process_qa_list(qa_list[:5])
with open(dataset_with_agent_answer, "w") as f:
    for qa in new_qa_list:
        f.write(json.dumps(qa) + "\n")

Loading video graph from /mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems/CZ_1/VzrB0Ns6bYo_30_5_-1_10_20_0.3_0.6.pkl
[]
Reasoning: The question asks about the cultural heritage of the Sung family. Currently, there is no knowledge provided about the Sung family or their cultural background. To answer this question, I need information about the Sung family's origins, traditions, or any explicit mention of their cultural heritage. Without this information, I cannot determine their cultural heritage.

[SEARCH] What is the cultural background or heritage of the Sung family?
[{'query': 'What is the cultural background or heritage of the Sung family?', 'retrieved memories': {'clip_0': ['<face_0>, <face_1>, and <character_10> pose in front of a Hello Kitty display at the airport.', '<character_10> is wearing sunglasses and makes a peace sign while in a car.', "<character_10> says, 'eating my way through Singapore.'", "<character_10> adds, 'Okay, I thought we were gonna eat light.'", 'A fam

Generating answers:   0%|          | 0/5 [00:00<?, ?it/s]

Loading video graph from /mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems/CZ_1/VzrB0Ns6bYo_30_5_-1_10_20_0.3_0.6.pkl
[]
[]
[]
[][]

Reasoning: The question asks whether the second lunch spot is an authentic Chinese restaurant. However, no knowledge has been provided about the second lunch spot, its cuisine, or any relevant details that could help determine its authenticity. To answer the question, I need information about the second lunch spot, such as its name, menu, reviews, or any description indicating its authenticity as a Chinese restaurant.

[SEARCH] Information about the second lunch spot, including its cuisine, name, and authenticity as a Chinese restaurant.
Reasoning: The question asks about a specific sauce mentioned by Isa that is paired with toast for breakfast and is popular in Singapore. Currently, there is no knowledge provided about Isa, the sauce, or any related context. To answer the question, I need information about Isa's mention of sauces, particularly one tha

Generating answers:  20%|██        | 1/5 [00:33<02:15, 33.95s/it]

Answer: Yes, the second lunch spot is an authentic Chinese restaurant, as indicated by the food, decor, and cultural elements such as the use of chopsticks, traditional dishes, and the ambiance resembling a casual Chinese dining experience.


Verifying answers: 100%|██████████| 5/5 [00:00<00:00,  6.06it/s]
